<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Fine-tuning a Large-Language Model</h1>

### Install Required packages
- [PyTorch](https://pytorch.org/get-started/locally/): Deep learning framework.
- Hugging Face Transformers(https://huggingface.co/docs/transformers/installation): To use pretrained state-of-the-art models.
- [Hugging Face Datasets](https://pypi.org/project/datasets/): Use public Hugging Face datasets
- [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html): For interactive notebook widgets

In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from model_constants import *
from model_train import retrain_model
from helper_funcs import *
import json
import uptrain
import torch
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
dtype = torch.float
device = torch.device("mps")

True
True


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
testing_text = "Nike shoes are very [MASK]."
original_model_outputs = test_model(model, testing_text)

In [3]:
def nike_text_present_func(inputs, outputs, gts=None, extra_args={}):
    is_present = []
    for input in inputs["text"]:
        this_present = "nike" in input.lower()
        is_present.append(bool(this_present))
    return is_present

uptrain_save_fold_name = "uptrain_smart_data_bert"
nike_text_present = uptrain.Signal("Nike Text Present", nike_text_present_func)

cfg = {
    'checks': [{
        'type': uptrain.Anomaly.EDGE_CASE,
        "signal_formulae": nike_text_present
    }],

    # Define where to save the retraining dataset
    'retraining_folder': uptrain_save_fold_name,
    
    # Define when to retrain, define a large number because we
    # are not retraining yet
    'retrain_after': 10000000000
}

framework = uptrain.Framework(cfg)

Deleting the folder:  uptrain_smart_data_bert


In [23]:
# raw_dataset = create_sample_dataset("raw_nike_reviews_data.json")
raw_dataset = create_sample_dataset("shoe_reviews_data.json")


with open(raw_dataset) as f:
    all_data = json.load(f)

for sample in all_data['data']:
    inputs = {'data': {'text': [sample['text']]}}
    framework.log(inputs = inputs, outputs = None)
# retraining_dataset = create_dataset_from_csv(uptrain_save_fold_name + "/1/smart_data.csv", "text", "retrain_dataset.json")

# raw_nike_reviews_dataset = create_dataset_from_csv("kaggle dataset/web_scraped.csv", "Content", "raw_nike_reviews_data.json")

shoe_reviews_dataset = create_dataset_from_csv("kaggle dataset/Shoes_Data.csv", "product_description", "shoe_reviews_data.json")


3050  edge cases identified out of  12197  total samples
3100  edge cases identified out of  12397  total samples
3150  edge cases identified out of  12597  total samples
3200  edge cases identified out of  12797  total samples
3250  edge cases identified out of  12997  total samples
3300  edge cases identified out of  13197  total samples
3350  edge cases identified out of  13397  total samples
3400  edge cases identified out of  13597  total samples
3450  edge cases identified out of  13797  total samples
3500  edge cases identified out of  13997  total samples
3550  edge cases identified out of  14197  total samples
3600  edge cases identified out of  14397  total samples
3650  edge cases identified out of  14597  total samples
3700  edge cases identified out of  14797  total samples
3750  edge cases identified out of  14997  total samples
3800  edge cases identified out of  15197  total samples
3850  edge cases identified out of  15397  total samples
3900  edge cases identified out

In [24]:
retrain_model(model, shoe_reviews_dataset)
retrained_model_outputs = test_model(model, testing_text)

Using custom data configuration default-fd5fcfc9d4747ef6


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/linuz/.cache/huggingface/datasets/json/default-fd5fcfc9d4747ef6/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/Users/linuz/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 103
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6


>>>Before training, Perplexity: 10.77


Epoch,Training Loss,Validation Loss
1,3.158700,2.925783
2,2.867500,2.609482
3,3.139300,2.642015


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12
  Batch size = 64


Training completed. Do not forget to shar

>>>After training, Perplexity: 10.52


In [36]:
test_model(model, 'shoes are [MASK]')

['.', ';', 'worn', '!', ':']

In [10]:
test_model(model, testing_text)

['popular', 'expensive', 'durable', 'common', 'comfortable']

In [11]:
original_model_outputs

['popular', 'expensive', 'durable', 'common', 'comfortable']